# Test02: test code for operations (linear,relu)


# Preparation 

## Preparation of google drive (if required)

In [1]:
#　colaboraory用: Google drive をマウントする
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# colaboratory用: フォルダを移る
%cd "drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


フォルダは自分の指定のものに変更して下さい。

In [3]:
%cd "ctorch220301/MiniCTorch_Prototype/notebook"

import sys
sys.path.append("../")

/content/drive/My Drive/Colab Notebooks/ctorch220301/MiniCTorch_Prototype/notebook


## Installation of MiniCTorch

In [4]:
! pip install lark-parser

     |████████████████████████████████| 103 kB 8.7 MB/s 


In [ ]:
! pip install git+https://github.com/kojima-r/MiniCTorch_Prototype.git

  Cloning https://github.com/kojima-r/MiniCTorch_Prototype.git to /tmp/pip-req-build-7qb_jvu6
  Running command git clone -q https://github.com/kojima-r/MiniCTorch_Prototype.git /tmp/pip-req-build-7qb_jvu6
  Created wheel for minictorch: filename=minictorch-0.0.1-py3-none-any.whl size=16013 sha256=f33bd3448954b186ffbca7356c2f7a4b886a532dca8bdb2b168fc9544660ee27
  Stored in directory: /tmp/pip-ephem-wheel-cache-clpogahs/wheels/1c/0a/b1/3618887a2db533ac9c7b971dc10589e3976893cdf9a74ca6da
Successfully built minictorch


## ニューラルネットの定義

In [5]:
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import minictorch

In [6]:
class Net( nn.Module ):
   def __init__(self):
      super(Net, self).__init__()
      self.fc1 = nn.Linear(2, 32)
      self.fc2 = nn.Linear(32, 32)
      self.fc3 = nn.Linear(32, 3)

   def forward(self, x):
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return x
      #return F.log_softmax(x, dim = 1)

プロジェクト名、作業フォルダ、JSONファイル名の設定

In [7]:
project = "test02"
folder = "./test"
json_path = folder + '/' + project +'.json'

In [8]:
! mkdir -p test

## 計算グラフをJSONファイルに出力する

In [9]:
def generate_json( json_path, input_x ): 
    model = Net()
    print(input_x)
    model.eval()
    with torch.no_grad():
        print("[JSON]", json_path )
        minictorch.trace( model, input_to_model, json_path )
    return model

In [10]:
torch.manual_seed( 1 )
input_to_model = torch.randn((1,2),requires_grad=True)

model = generate_json( json_path, input_to_model )

tensor([[0.6614, 0.2669]], requires_grad=True)
[JSON] ./test/test02.json
skip: Net/Linear[fc1]/weight/fc1
skip: Net/Linear[fc1]/weight/fc1
skip: Net/Linear[fc2]/weight/fc2
skip: Net/Linear[fc2]/weight/fc2
skip: Net/Linear[fc3]/weight/fc3
skip: Net/Linear[fc3]/weight/fc3


JSONファイルをC++コードに変換する

In [11]:
minictorch.convert_all( project, folder, model, json_path, input_to_model, {}, code="test" )

[JSON] ./test/test02.json
param: xin  - str loop  0  /  0
{'name': 'Net/Linear[fc1]/weight/weight.7', 'op': 'prim::GetAttr', 'in': [], 'output_id': 0, 'shape': [], 'out': [3], 'sorted_id': 1}
param: fc1_weight  - str loop  7  /  7
{'name': 'Net/Linear[fc1]/bias/bias.7', 'op': 'prim::GetAttr', 'in': [], 'output_id': 0, 'shape': [], 'out': [3], 'sorted_id': 2}
param: fc1_bias  - str loop  3  /  3
{'name': 'Net/Linear[fc2]/weight/weight.9', 'op': 'prim::GetAttr', 'in': [], 'output_id': 0, 'shape': [], 'out': [7], 'sorted_id': 5}
param: fc2_weight  - str loop  127  /  127
{'name': 'Net/Linear[fc2]/bias/bias.9', 'op': 'prim::GetAttr', 'in': [], 'output_id': 0, 'shape': [], 'out': [7], 'sorted_id': 6}
param: fc2_bias  - str loop  3  /  3
{'name': 'Net/Linear[fc3]/weight/weight', 'op': 'prim::GetAttr', 'in': [], 'output_id': 0, 'shape': [], 'out': [11], 'sorted_id': 9}
param: fc3_weight  - str loop  11  /  11
{'name': 'Net/Linear[fc3]/bias/bias', 'op': 'prim::GetAttr', 'in': [], 'output_id': 

pythonコードを実行する

In [12]:
# check code
model.train()
yy = model( input_to_model )
print(yy)
yy.sum().backward()
print(input_to_model.grad)

tensor([[ 0.1536, -0.1925,  0.0064]], grad_fn=<AddmmBackward0>)
tensor([[-0.1284,  0.1121]])


## 環境構築
(注意) インストール済みの場合は各自の環境に合わせて変更して下さい。

In [ ]:
! git clone https://github.com/xtensor-stack/xtensor.git
! git clone https://github.com/xtensor-stack/xtensor-blas.git
! git clone https://github.com/xtensor-stack/xtl.git
! git clone https://github.com/nlohmann/json.git


In [ ]:
! git clone https://github.com/kojima-r/MiniCTorch_Prototype.git

C++コードをコンパイル、リンクする

In [14]:
C_SRC = './test/test02.cpp ./test/test02_param.cpp'
C_OPT = '-I ../src -I ./MiniCTorch_Prototype/src -I ./xtensor/include -I ./xtensor-blas/include -I ./xtl/include -I ./json/include '
C_OUT = './test/test02'
!echo "SRC = "$C_SRC
!echo "OPT = "$C_OPT
!echo "OUT = "$C_OUT
!g++ -std=c++14 $C_SRC $C_OPT -lcblas -o $C_OUT

SRC = ./test/test02.cpp ./test/test02_param.cpp
OPT = -I ../src -I ./MiniCTorch_Prototype/src -I ./xtensor/include -I ./xtensor-blas/include -I ./xtl/include -I ./json/include
OUT = ./test/test02


In [15]:
! ./test/test02

### forward computation ...
{{ 0.153593, -0.192527,  0.006398}}
### backward computation ...
input_grad{{-0.128356,  0.112093}}
